<a href="https://colab.research.google.com/github/Theallkeeeymist/sentiment-analysis/blob/main/Sentiment_analysis_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

Data Acquisition and convertion of sentiment into 1 and 0

In [2]:
data=pd.read_csv("/content/IMDB Dataset.csv")

In [3]:
data.shape

(50000, 2)

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
data['sentiment']=data['sentiment'].map({'positive':1, 'negative':0})

In [6]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
(data['sentiment']==1).sum()

np.int64(25000)

Text Preprocessing

In [8]:
!pip install beautifulsoup4

In [9]:
from bs4 import BeautifulSoup
import re

In [10]:
def preprocess(review):
  review=review.lower()
  review=BeautifulSoup(review,"html.parser").get_text()
  review=re.sub(r'[^a-zA-Z]', ' ', review)

  return review

In [11]:
data["clean_review"]=data['review'].apply(preprocess)

In [12]:
data["clean_review"][0]

'one of the other reviewers has mentioned that after watching just   oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [49]:
tokenizer=Tokenizer(num_words=10000, oov_token="<oov>")
tokenizer.fit_on_texts(data['clean_review'])

seq=tokenizer.texts_to_sequences(data['clean_review'])

padded_sequences = pad_sequences(seq, padding='post', truncating='post', maxlen=500)

In [57]:
word_index=tokenizer.word_index
total_vocab=len(word_index)
print(f"vocab of dataset: {total_vocab}")

vocab of dataset: 101400


In [50]:
padded_sequences

array([[  29,    5,    2, ...,    0,    0,    0],
       [   4,  391,  121, ...,    0,    0,    0],
       [  10,  192,   11, ...,    0,    0,    0],
       ...,
       [  10,  231,    4, ...,    0,    0,    0],
       [  10,  139,  168, ...,    0,    0,    0],
       [  57,   29, 5819, ...,    0,    0,    0]], dtype=int32)

In [51]:
labels=data['sentiment']

In [52]:
labels

,sentiment
0,1
1,1
2,1
3,0
4,1
...,...
49995,1
49996,0
49997,0
49998,0


In [53]:
from sklearn.model_selection import train_test_split
#
X_train,X_test,y_train,y_test=train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Bidirectional,LSTM,Embedding,Dense,Dropout, SimpleRNN

model=Sequential()
model.add(Embedding(input_dim=10000, output_dim=64, input_shape=(500,)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ (None, 500, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_27                │ (None, 500, 256)       │       197,632 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 500, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_28                │ (None, 128)            │       164,352 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,002,113 (3.82 MB)

 Trainable params: 1,002,113 (3.82 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
from tensorflow.keras.callbacks import Callback

class MetricsCallback(Callback):
    def __init__(self, val_data):
        super().__init__()
        self.val_data = val_data

    def on_epoch_end(self, epoch, logs=None):
        val_X, val_y = self.val_data
        val_preds = (self.model.predict(val_X) > 0.5).astype(int)

        f1 = f1_score(val_y, val_preds)
        precision = precision_score(val_y, val_preds)
        recall = recall_score(val_y, val_preds)

        print(f"\n📊 Epoch {epoch+1} — Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")

In [63]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
metrics_callback = MetricsCallback(val_data=(X_test, y_test))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stop, metrics_callback])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step

📊 Epoch 1 — Precision: 0.7406 | Recall: 0.4828 | F1: 0.5846
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 87s 84ms/step - accuracy: 0.5498 - loss: 0.6876 - val_accuracy: 0.6557 - val_loss: 0.6298
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step

📊 Epoch 2 — Precision: 0.9208 | Recall: 0.5356 | F1: 0.6773
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 86s 86ms/step - accuracy: 0.6962 - loss: 0.5885 - val_accuracy: 0.7355 - val_loss: 0.5500
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step

📊 Epoch 3 — Precision: 0.9121 | Recall: 0.8154 | F1: 0.8611
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 140s 84ms/step - accuracy: 0.8235 - loss: 0.4105 - val_accuracy: 0.8650 - val_loss: 0.3101
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step

📊 Epoch 4 — Precision: 0.8721 | Recall: 0.9198 | F1: 0.8953
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 144s 86ms/step - accuracy: 0.9047 - loss: 0.2514 - val_accuracy: 0.8879 - val_loss: 0.2813
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step

📊 Epoch 5

['model.pkl']